In [113]:
import os
import json

In [114]:
import pandas as pd

In [115]:
def text_to_jsonl(path):
    filename,_ = path.split(".")
    data = open(path).read()
    temp = data.split("""{\n  "questions":""")

    to_json = []
    for Q_A in temp[1:]:
       try:
          response = json.loads("""{\n  "questions":"""+Q_A)
          to_json.append(response)
       except:
          continue

    with open(f'{filename}.jsonl', 'w') as jsonl_file:
        try:
            for item in to_json:
                for q in item['questions']:
                    json.dump({"question": q["question"], "answer": q["answer"]}, jsonl_file)
                    jsonl_file.write('\n')
        except:
            pass

In [116]:
#text_to_jsonl("temp_mixtral.txt")
#text_to_jsonl("temp_llama.txt")

In [117]:
def count_words(text):
    return len(text.split())

In [118]:
def check_filter_words(word):
  if word in ['context','question','answer']:
    return False
  else:
    return True

In [119]:
def load_jsonl(file_path):
    data = []
    with open(file_path, 'r') as file:
        for line in file:
            data.append(json.loads(line))
    return data

In [107]:
#data_mixtral = pd.DataFrame(load_jsonl('temp_mixtral.jsonl'))
#data_llama = pd.DataFrame(load_jsonl('temp_llama.jsonl'))

In [120]:
df = pd.read_csv("combined_data.csv")

In [121]:
df.shape

(4058, 2)

In [122]:
df.head()

,question,answer
0,Why are there still significant travel delays ...,There are still significant travel delays and ...
1,What challenges are people in New Jersey facin...,People in New Jersey are facing significant ch...
2,How are the two major candidates in the Iowa c...,"With one week to go before the Iowa caucuses, ..."
3,How has Donald Trump's loyalty been questioned...,Donald Trump's loyalty has been questioned in ...
4,What two different methods are being used in p...,The two different methods being used in polls ...


In [123]:
average_word_count = df['answer'].apply(count_words).mean()
average_word_count

41.68753080335141

In [135]:
keywords = ['context',"provided context","mentioned in the context"]

In [136]:
filtered_rows = []
for index, row in df.iterrows():
    if any(keyword in row['question'].lower() or keyword in row['answer'].lower() for keyword in keywords):
        filtered_rows.append(row)

filtered_df = pd.DataFrame(filtered_rows)

In [138]:
filtered_df.iloc[1].answer

'According to a recent poll, 50% of the French population believes that they should not deal with a country like Iran due to its poor human rights record. However, the other half of the population appears to be more pragmatic, considering the potential benefits of these ties.'

In [142]:
final_df = df.copy()
for word in keywords:
    final_df = final_df[~(final_df['answer'].str.contains(word, case=False))]

In [143]:
final_df.shape

(3972, 2)

In [144]:
final_df.to_csv('final_data.csv', index=False)